In [1]:
import os, random, time
import cv2, imutils
from skimage.measure import compare_ssim
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
%cd '/Users/sharan_srinivasan/Documents/adhoc/kilaigal'

/Users/sharan_srinivasan/Documents/adhoc/kilaigal


1. Read a sample video stream
2. Fetch base frame,
3. Diff generator --> Identify motion changes, record ts and callback to chunk display
4. Process frames : BW,Guassian ablation, thresholding, countour detection --> Object localization. Callback to localized object image saver
5. end stream and output ts chunks. 
6. Try to output one frame of the localized object

In [3]:

# check the framedelta's rmse 
def compute_rmse_from_image_diff(imageDiff):
    diffSum = np.sum(imageDiff.astype('float')**2)
    diffCardinality = (imageDiff.shape[0]*imageDiff.shape[1])
    rmse = np.sqrt(diffSum/ diffCardinality)
    return rmse

# Check SSIM
def compute_structural_similarity(imgage, reference):
    s = compare_ssim(imgage, reference)
    return s


In [4]:
# Read the file and get an initial frame
vid = cv2.VideoCapture('videos/Water Drop-Small.mp4')

In [5]:
def simplePlay():
    cap = cv2.VideoCapture('videos/Water Drop-Small.mp4')

    while(cap.isOpened()):
        ret, frame = cap.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #     gray = frame

        cv2.imshow('frame',gray)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break

    cap.release()
    cv2.destroyAllWindows()

In [19]:
def preProcessImageFrame(frame):
    # Resize the image so it plays nicely with openCV
    frame = imutils.resize(frame, width=512, height=256)
    # Convert it to grayscale to enable simple diffing
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Add some blur to smooth and get rid of noise
    frame = cv2.blur(frame, (1, 1), 0)
    return frame

In [28]:
all_motion_sequences = []
current_motion_seq = []
cap = cv2.VideoCapture('videos/flower_small.mp4')
baseFrame = None
SSIM_DISTANCE_THRESHOLD = 0.1
frame_num = 0 # codex frame index
motion_state = 0 # Whether the video is in motion


def fetch_frame_with_lag():    
    global frame_num
    i = frame_num
    while i < frame_num + 10:
        ret, nextFrame = cap.read()
        if ret == False:
            break;
        i+=1
    frame_num = i
    return ret, nextFrame

"""
    We update the base frame to adapt to the new normalcy. The intuition
    is that the over time the current frame will dampen precious base.
    X = (\alpha)X + (1-\alpha)C
    Where 
        - alpha : Affine smoothing weight
        - X: Existing base frame
        - C: Current frame
"""
def updateBaseFrameWithSmoothing(baseFrame, curFrame):
    #print("baseFrame", baseFrame.shape)
    #print("curFrame", curFrame.shape)
    alpha = 1.0
    newBaseFrame = alpha*baseFrame + (1-alpha)*curFrame
    # print("newBaseFrame", newBaseFrame.shape)
    return newBaseFrame.astype(int)
    

while(cap.isOpened()):
    # Get the current frame and preprocess it
    ret, frame = fetch_frame_with_lag()
    if(ret == False):
        if(motion_state == 1):
            print("Motion sequence ended at frame {}. Resetting State".format(current_motion_seq))
            all_motion_sequences.append(current_motion_seq)
            current_motion_seq = []
            motion_state = 0
        print("Video stream not recieved. Aborting ...")
        break;
        
    ppFrame = preProcessImageFrame(frame)

    if baseFrame is None:
        baseFrame = ppFrame.copy()
        continue
    else:
        # update the base frame
        alpha = 1.0
#         newBaseFrame = np.array(alpha*baseFrame + (1-alpha)*ppFrame).astype(int)
        newBaseFrame = cv2.addWeighted(baseFrame, 0.9, ppFrame, 0.1, 0, dtype = cv2.CV_32F)
        # baseFrame = newBaseFrame.copy()
        
    # Determine if there is a motion sequence and update state
    ssim = 1 - compute_structural_similarity(baseFrame, ppFrame)
    if(ssim > SSIM_DISTANCE_THRESHOLD):
        current_motion_seq.append(frame_num)
        if(motion_state == 0):
            print("Motion sequence detected on frame {} with ssim distance {}".format(frame_num, ssim))
            motion_state = 1
            baseFrame = np.copy(ppFrame)
    else:
        # When there is no motion, reset the state and end the motion seqeuence
        if(motion_state == 1):
            print("Motion sequence ended at frame {}. Resetting State".format(current_motion_seq))
            all_motion_sequences.append(current_motion_seq)
            current_motion_seq = []
            motion_state = 0
    
    frame_num +=1
    
    # To localize the Image
#     Get a simple diff with the base frame
    frameDelta = cv2.absdiff(baseFrame, ppFrame)
    # threshold the delta
    frameDeltaThresholded = cv2.threshold(frameDelta, 8, 255, cv2.THRESH_BINARY)
    
    # play the image
    cv2.imshow('frame',ppFrame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

cap.release()
cv2.destroyAllWindows()

Motion sequence detected on frame 53 with ssim distance 0.173420516095266
Motion sequence ended at frame [53]. Resetting State
Motion sequence detected on frame 75 with ssim distance 0.1179341543556599
Motion sequence ended at frame [75]. Resetting State
Motion sequence detected on frame 119 with ssim distance 0.12073347389465305
Motion sequence ended at frame [119]. Resetting State
Motion sequence detected on frame 163 with ssim distance 0.15018130303404542
Motion sequence ended at frame [163]. Resetting State
Motion sequence detected on frame 185 with ssim distance 0.13546282006911625
Motion sequence ended at frame [185]. Resetting State
Motion sequence detected on frame 218 with ssim distance 0.18795790591263695
Motion sequence ended at frame [218]. Resetting State
Motion sequence detected on frame 240 with ssim distance 0.13787667216866029
Motion sequence ended at frame [240]. Resetting State
Motion sequence detected on frame 273 with ssim distance 0.12175489350071944
Motion sequen

In [30]:
def compress_all_sequences(all_motion_sequences):
    '''
    Takes a function of motion sequences and return (start_time) tuples.
    
    :param all_motion_sequences: A list of lists, with each list being a motion sequence
    :returns: A sequence of tuples
    '''
    
    compressed_sequences = []
    for s in all_motion_sequences:
        t = (s[0])
        compressed_sequences.append(t)
    return compressed_sequences
    
print(compress_all_sequences(all_motion_sequences))

[53, 75, 119, 163, 185, 218, 240, 273, 317, 405, 449, 526, 570, 603, 658]


In [18]:
?cv2.addWeighted